In [1]:
from ase.build import graphene_nanoribbon
import nglview as nv

In [61]:
def visualize_nanoribbon(nanoribbon):
    import nglview as nv
    import numpy as np

    w = nv.show_ase(nanoribbon)
    w.add_ball_and_stick()
    w.add_label(color="black", labelType="atomindex")

    origin = nanoribbon.positions.min(axis=0) - np.array([2.0, 2.0, 2.0])
    L = 2.0

    # стрелки
    w.shape.add_arrow(origin, origin + [L, 0, 0], [1, 0, 0], 0.2)  # X (red)
    w.shape.add_arrow(origin, origin + [0, L, 0], [0, 1, 0], 0.2)  # Y (green)
    w.shape.add_arrow(origin, origin + [0, 0, L], [0, 0, 1], 0.2)  # Z (blue)

    # подписи
    w.shape.add_text(origin + [L, 0, 0], [1, 0, 0], 1.0, "X")
    w.shape.add_text(origin + [0, L, 0], [0, 1, 0], 1.0, "Y")
    w.shape.add_text(origin + [0, 0, L], [0, 0, 1], 1.0, "Z")

    return w

In [71]:
def neighbor_list(nanoribbon, cutoff=1.85):
    from ase.neighborlist import neighbor_list

    i, j, S = neighbor_list('ijS', nanoribbon, cutoff)
    cell = nanoribbon.cell.array
    pos = nanoribbon.positions

    neigh = {a: [] for a in range(len(nanoribbon))}
    for ia, ja, Sa in zip(i, j, S):
        rj_img = pos[ja] + Sa @ cell  # координаты соседа с учётом трансляции
        neigh[int(ia)].append((int(ja), tuple(int(x) for x in Sa), rj_img))

    return neigh

In [ ]:
# Построение наноленты

nanoribbon = graphene_nanoribbon(
    3, 3,
    type='zigzag',      # или 'armchair'
    saturated=False,    # пассивация водородом по краям
    C_H=1.09,
    C_C=1.42,
    vacuum=None,
    magnetic=False,
    initial_mag=1.12,
    sheet=False,
    main_element='C',
    saturate_element='H'
)

In [73]:
visualize_nanoribbon(nanoribbon)

NGLWidget()

In [77]:
neigh = neighbor_list(nanoribbon)
pos = nanoribbon.positions

for ia in range(len(nanoribbon)):
    print(f"Atom {ia:3d}  r=({pos[ia][0]:8.4f},{pos[ia][1]:8.4f},{pos[ia][2]:8.4f})  neighbors={len(neigh[ia])}")
    for ja, Sa, rj in neigh[ia]:
        print(f"   -> j={ja:3d}, S={Sa}, rj=({rj[0]:8.4f},{rj[1]:8.4f},{rj[2]:8.4f})")

Atom   0  r=(  0.0000,  0.0000,  0.0000)  neighbors=2
   -> j=  1, S=(0, 0, 0), rj=(  0.7100,  0.0000,  1.2298)
   -> j=  5, S=(0, 0, -1), rj=(  0.7100,  0.0000, -1.2298)
Atom   1  r=(  0.7100,  0.0000,  1.2298)  neighbors=3
   -> j=  2, S=(0, 0, 0), rj=(  0.0000,  0.0000,  2.4595)
   -> j=  0, S=(0, 0, 0), rj=(  0.0000,  0.0000,  0.0000)
   -> j=  6, S=(0, 0, 0), rj=(  2.1300,  0.0000,  1.2298)
Atom   2  r=(  0.0000,  0.0000,  2.4595)  neighbors=2
   -> j=  3, S=(0, 0, 0), rj=(  0.7100,  0.0000,  3.6893)
   -> j=  1, S=(0, 0, 0), rj=(  0.7100,  0.0000,  1.2298)
Atom   3  r=(  0.7100,  0.0000,  3.6893)  neighbors=3
   -> j=  8, S=(0, 0, 0), rj=(  2.1300,  0.0000,  3.6893)
   -> j=  2, S=(0, 0, 0), rj=(  0.0000,  0.0000,  2.4595)
   -> j=  4, S=(0, 0, 0), rj=(  0.0000,  0.0000,  4.9190)
Atom   4  r=(  0.0000,  0.0000,  4.9190)  neighbors=2
   -> j=  3, S=(0, 0, 0), rj=(  0.7100,  0.0000,  3.6893)
   -> j=  5, S=(0, 0, 0), rj=(  0.7100,  0.0000,  6.1488)
Atom   5  r=(  0.7100,  0.0000,  

In [74]:
# Можно руками задать периодические граничные условия вдоль определенных осей
nanoribbon.pbc = (False, False, False)  # отключение трансляции по оси x

In [75]:
neigh = neighbor_list(nanoribbon)
pos = nanoribbon.positions

for ia in range(len(nanoribbon)):
    print(f"Atom {ia:3d}  r=({pos[ia][0]:8.4f},{pos[ia][1]:8.4f},{pos[ia][2]:8.4f})  neighbors={len(neigh[ia])}")
    for ja, Sa, rj in neigh[ia]:
        print(f"   -> j={ja:3d}, S={Sa}, rj=({rj[0]:8.4f},{rj[1]:8.4f},{rj[2]:8.4f})")

Atom   0  r=(  0.0000,  0.0000,  0.0000)  neighbors=1
   -> j=  1, S=(0, 0, 0), rj=(  0.7100,  0.0000,  1.2298)
Atom   1  r=(  0.7100,  0.0000,  1.2298)  neighbors=3
   -> j=  2, S=(0, 0, 0), rj=(  0.0000,  0.0000,  2.4595)
   -> j=  0, S=(0, 0, 0), rj=(  0.0000,  0.0000,  0.0000)
   -> j=  6, S=(0, 0, 0), rj=(  2.1300,  0.0000,  1.2298)
Atom   2  r=(  0.0000,  0.0000,  2.4595)  neighbors=2
   -> j=  1, S=(0, 0, 0), rj=(  0.7100,  0.0000,  1.2298)
   -> j=  3, S=(0, 0, 0), rj=(  0.7100,  0.0000,  3.6893)
Atom   3  r=(  0.7100,  0.0000,  3.6893)  neighbors=3
   -> j=  8, S=(0, 0, 0), rj=(  2.1300,  0.0000,  3.6893)
   -> j=  2, S=(0, 0, 0), rj=(  0.0000,  0.0000,  2.4595)
   -> j=  4, S=(0, 0, 0), rj=(  0.0000,  0.0000,  4.9190)
Atom   4  r=(  0.0000,  0.0000,  4.9190)  neighbors=2
   -> j=  3, S=(0, 0, 0), rj=(  0.7100,  0.0000,  3.6893)
   -> j=  5, S=(0, 0, 0), rj=(  0.7100,  0.0000,  6.1488)
Atom   5  r=(  0.7100,  0.0000,  6.1488)  neighbors=2
   -> j= 10, S=(0, 0, 0), rj=(  2.1300

Попытка в хиральные ленты

In [108]:
import numpy as np
from ase import Atoms

try:
    from ase.build import make_supercell
except Exception:
    from ase.build.supercells import make_supercell

from ase.build import graphene


def _graphene_primitive(a=2.46) -> Atoms:
    try:
        return graphene(a=a, size=(1, 1, 1), vacuum=0.0)
    except TypeError:
        try:
            return graphene(latticeconstant={"a": a}, size=(1, 1, 1), vacuum=0.0)
        except TypeError:
            return graphene(latticeconstant=a, size=(1, 1, 1), vacuum=0.0)


def chiral_gnr_nm_no_cut(
    n: int, m: int,
    *,
    Nx: int = 8,     # число периодов вдоль оси (длина)
    Ny: int = 4,     # число периодов по ширине
    a: float = 2.46, # параметр решетки графена (Å)
    vacuum: float = 10.0,
    pbc_x: bool = True,
) -> Atoms:
    """
    Хиральная нанолента :
    - C = n a1 + m a2
    - ширина задаётся целочисленным перпендикуляром W = p a1 + q a2
    - повторяем Nx, Ny
    - ортонормально поворачиваем так, чтобы ось ленты = X (без искажений)
    """

    if n == 0 and m == 0:
        raise ValueError("(n,m)=(0,0) не задаёт направление.")

    prim = _graphene_primitive(a=a)

    # Векторы решётки
    a1 = np.array(prim.cell[0]); a1[2] = 0.0
    a2 = np.array(prim.cell[1]); a2[2] = 0.0

    # Ось
    C = n * a1 + m * a2

    # Целочисленный перпендикуляр к C: p(2n+m) + q(n+2m) = 0
    g = np.gcd(n + 2*m, 2*n + m)
    p = (n + 2*m) // g
    q = -(2*n + m) // g
    W = p * a1 + q * a2

    # Суперячейка: [C, W] в базисе (a1,a2)
    S = np.array([[n, p, 0],
                  [m, q, 0],
                  [0, 0, 1]], dtype=int)

    cell_nm = make_supercell(prim, S)

    # Лента конечной ширины: повторяем Ny раз по W и выключаем PBC по y
    rib = cell_nm.repeat((Nx, Ny, 1))
    rib.set_pbc([pbc_x, False, False])

    # --- Ортонормальное выравнивание 
    ex = C / np.linalg.norm(C)

    Wy = W - np.dot(W, ex) * ex
    ey = Wy / np.linalg.norm(Wy)

    # ez - перпендикуляр к плоскости
    ez = np.array([0.0, 0.0, 1.0])

    pos = rib.get_positions()
    x = pos @ ex
    y = pos @ ey
    z = pos @ ez

    # Сдвигаем так, чтобы min=0
    x -= x.min()
    y -= y.min()

    rib.set_positions(np.column_stack([x, y, z]))

    # Ячейка: прямоугольная (для удобства, геометрию не ломает)
    Lx = x.ptp() + 2.0
    Ly = y.ptp() + 2.0 * vacuum
    Lz = 2.0 * vacuum

    rib.set_cell([[Lx, 0, 0],
                  [0,  Ly, 0],
                  [0,  0,  Lz]])
    rib.center(axis=(1, 2))

    return rib

rib = chiral_gnr_nm_no_cut(3, 1, Nx=1, Ny=1, vacuum=10.0, pbc_x=True)
print("atoms:", len(rib))

atoms: 52


In [109]:
visualize_nanoribbon(rib)

NGLWidget()

In [107]:
neigh = neighbor_list(rib)
pos = rib.positions

for ia in range(len(nanoribbon)):
    print(f"Atom {ia:3d}  r=({pos[ia][0]:8.4f},{pos[ia][1]:8.4f},{pos[ia][2]:8.4f})  neighbors={len(neigh[ia])}")
    for ja, Sa, rj in neigh[ia]:
        print(f"   -> j={ja:3d}, S={Sa}, rj=({rj[0]:8.4f},{rj[1]:8.4f},{rj[2]:8.4f})")

Atom   0  r=(  0.0000, 18.3206, 10.0000)  neighbors=1
   -> j=  1, S=(0, 0, 0), rj=(  1.3947, 18.0522, 10.0000)
Atom   1  r=(  1.3947, 18.0522, 10.0000)  neighbors=3
   -> j=  0, S=(0, 0, 0), rj=(  0.0000, 18.3206, 10.0000)
   -> j= 16, S=(0, 0, 0), rj=(  1.8596, 16.7102, 10.0000)
   -> j= 14, S=(0, 0, 0), rj=(  2.3245, 19.1259, 10.0000)
Atom   2  r=(  5.1139, 33.6199, 10.0000)  neighbors=1
   -> j=  3, S=(0, 0, 0), rj=(  6.5085, 33.3515, 10.0000)
Atom   3  r=(  6.5085, 33.3515, 10.0000)  neighbors=2
   -> j= 18, S=(0, 0, 0), rj=(  6.9734, 32.0095, 10.0000)
   -> j=  2, S=(0, 0, 0), rj=(  5.1139, 33.6199, 10.0000)
Atom   4  r=(  4.6490, 31.2042, 10.0000)  neighbors=2
   -> j=  5, S=(0, 0, 0), rj=(  6.0437, 30.9358, 10.0000)
   -> j= 43, S=(0, 0, 0), rj=(  3.7192, 30.1306, 10.0000)
Atom   5  r=(  6.0437, 30.9358, 10.0000)  neighbors=3
   -> j= 20, S=(0, 0, 0), rj=(  6.5085, 29.5938, 10.0000)
   -> j= 18, S=(0, 0, 0), rj=(  6.9734, 32.0095, 10.0000)
   -> j=  4, S=(0, 0, 0), rj=(  4.6490